<a href="https://colab.research.google.com/github/dhayanesh/bert-embeddingExtraction/blob/main/bert_embeddingsExtraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers


In [ ]:
import torch
from transformers import BertTokenizer, BertForSequenceClassification, AdamW

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased')
optimizer = AdamW(model.parameters(), lr=1e-5)

#reusing code for extracting updated indices from DLRM implementation
def get_nonzero_grad_indices(embedding_layer):
    nonzero_grad_indices = []

    def hook(grad):
        if grad.is_sparse:
            grad = grad.to_dense()
        unique_nonzero_indices = torch.unique(torch.nonzero(grad, as_tuple=False)[:, 0])
        nonzero_grad_indices.extend(unique_nonzero_indices.tolist())

    handle = embedding_layer.weight.register_hook(hook)
    return nonzero_grad_indices, handle

embedding_layer = model.bert.embeddings.word_embeddings
nonzero_grad_indices, handle = get_nonzero_grad_indices(embedding_layer)

for i in range(10):
    input_text = "Bidirectional Encoder Representations from Transformers (BERT) is a family of language models introduced by researchers at Google. BERT is based on the transformer architecture. Specifically, BERT is composed of Transformer encoder layers"
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=512)

    labels = torch.tensor([1]).unsqueeze(0)
    outputs = model(**inputs, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

updated_embedding_values = {}
with torch.no_grad():
    for index in nonzero_grad_indices:
        updated_embedding = embedding_layer.weight.data[index].tolist()
        key = 'bert.embeddings.word_embeddings.weight'
        if key not in updated_embedding_values:
            updated_embedding_values[key] = {}
        updated_embedding_values[key][index] = updated_embedding

print(updated_embedding_values)
handle.remove()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'bert.embeddings.word_embeddings.weight': {101: [0.01358626876026392, -0.02653452195227146, -0.02346549928188324, -0.007745502050966024, 0.008578384295105934, -0.007706059608608484, -0.009840300306677818, 0.006061703432351351, 0.004648061469197273, -0.031027479097247124, 0.018927671015262604, -0.006051891483366489, -0.016695106402039528, 0.011725081130862236, -0.036288339644670486, 0.00839014258235693, -0.001175550278276205, 0.010278785601258278, 0.016737496480345726, -0.030312540009617805, -0.012414420023560524, -0.0252163577824831, -0.008916929364204407, 0.008242282085120678, -0.019965991377830505, -0.015861837193369865, -0.003800701117143035, 0.0013804931659251451, 0.007004215382039547, 0.0016534782480448484, -0.002833814825862646, 0.009451971389353275, -0.022726405411958694, 0.019359637051820755, -0.013393023051321507, -0.023812731727957726, -0.014659016393125057, 0.009731956757605076, -0.0022860963363200426, 0.030604226514697075, 0.006738347001373768, -0.0026943720877170563, -0.0